# Lab 4 - InSAR
As we have seen from the videos and lectures, SAR (Synthetic Aperature RADAR) is a widely-used remote sensing technique for measuring a range of surface properties. This class is about geodetic-type remote sensing measurements, so we will focus on InSAR, or Interferometric SAR. In this lab you will gain some experience accessing, modeling, and interpreting InSAR data. 

In [ ]:
# some imports
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

## Part 1 - Modeling an earthquake using Visible Earthquakes 
In this part of the lab you will use a website called __[Visible Earthquakes](https://earthquakes.aranzgeo.com/)__ to model a normal-faulting earthquake that happened in Turkey in 1995, as well as a second earthquake of your choice. Navigate to the webpage and go through the __[tutorial](https://earthquakes.aranzgeo.com/start)__ on how to use the site. Then navigate back to the home page and select the **Dinar Earthquake**. You can use both the wrapped and unwrapped versions of the data to help you determine the best parameters for the event. Once you have your best-fitting model, put the results in the cell below. Make sure to match the parameter values to the order given in the comment. After completing the assignment for the Dinar earthquake, pick another earthquake of your choice and follow the same procedure. Be sure to mark which earthquake you did and what parameters you chose!

In [ ]:
# param_dinar = np.array([strike, dip, rake, length,slip,depth,center,magnitude])
param_dinar = None # add the parameters of your best-fit model

# Second earthquake
eq_name = None  # add the name of the second earthquake you did. 
param_second = None # add the parameters of your best-fit model

# Part 2 - Accessing and analyzing InSAR data

## InSAR Data sources
There are several places and ways to access InSAR data. Some data is available directly as InSAR: SAR products that have already been pre-processed and interfered to produce an interferogram, two SAR acquisitions that have been differenced. 

###__[1. Alaska Satellite Facility](https://asf.alaska.edu/)__
The __[ASF search page](https://search.asf.alaska.edu/#/)__ interactive webpage allows you to select a region or specify a bounding box, select which products you are interested in, download individual products, or perform a bulk download using a Python script. This site is most useful if you will processing SAR data into InSAR products, but it also contains the ARIA interferograms that we will use in this lesson. This site has SAR/InSAR data available from several different missions, including RADARSAT-1, Sentinel-1, JPL's UAVSAR, ALOS-PALSAR, and some ERS data.
<img src="https://raw.githubusercontent.com/jlmaurer/GE6146/master/notebooks/images/Vertex_homescreen.png" alt="Vertex home page" width="700">

### __[2. ARIA](https://aria.jpl.nasa.gov/)__
The ARIA project is "a collaboration between JPL and Caltech to exploit radar and optical remote sensing, GPS, and seismic observations for hazard science and response." They will eventually provide automatic and real-time processing of SAR, InSAR, and GNSS data to aid in rapid response to all kinds of earth hazards, including earthquakes, volcanoes, large landslides, flooding, storms, etc. Accessing InSAR data using ARIA can be done through ASF or through the __[ARIA-tools](https://github.com/aria-tools/ARIA-tools)__ API, which is what we will use in this lab. 
<img src="https://raw.githubusercontent.com/jlmaurer/GE6146/master/notebooks/images/ARIA_homepage.png" alt="ARIA project home page" width="700">

### __[3. UAVSAR](https://uavsar.jpl.nasa.gov/)__
JPL's airborne SAR sensor is called "UAVSAR" for "Uninhabited Aeriel Vehicle SAR" (although the vehicle almost always actually inhabited!). This mission collects airborne SAR data and makes interferograms available on the __[project website](https://uavsar.jpl.nasa.gov/)__. On their website you can search for and download data for the specific areas where it has been acquired; unlike satellite-based sensors, UAVSAR is on-demand, so it is not acquired everywhere nor all the time. 

## Processing Workflow Considerations
How you process the data to get a result that can be used for geodetic purposes depends on what data you access; i.e.,  Single-Look complex (SLC) products in radar coordinates versus geocoded interferograms. The process you will want to use will depend on several factors, including: 
1. What data is available, and in what format. This includes:  
   - Does the data you want exist?
   - Has it been processed already, and to what level? 
   - Do you need a single pair (interferogram), or do you need to look at time-series?
   - Is the data publically available or will you need to access it through other means (e.g. writing a grant)
   - Does the data exist at the necessary resolution, temporal sampling, etc. for your application?
2. What processing algorithms you have access to use or know how to use?  
   - Some algorithms are open-source and cross-platform, others must be paid for or only work on one OS
   - Some algoirthms are simple and do a few things, others are complex and do many things
   - How precise do your measurements need to be? If your signal is large, you can use simpler processing techniques and still get a useful signal, but if your signal is small you will need to use the best possible methods to perform your analysis. 
3. Do you need time-series analysis or single event pairs? 
4. What computational capabilities (computers, servers, memory, disk space) do you need and do you have access to it?  
   - SAR/InSAR scenes can be large, and processing several can be very computational intensive. 
   - You will need **large memory, processing power, and storage space** capabilities
   - In most cases, access to specialized equipment like many processors or GPUs can significantly aid computational capabilities.  
5. There are many software tools and cutting-edge advanced processing techniques that we will not delve into in this class, but may be useful to you for futher research. Note that some of these are research-grade codes or otherwise not well developed, so learning to use them could take some significant time and effort. Some of these are:  
   - __[The Sentinel-1 toolbox, or SNAP](https://sentinel.esa.int/web/sentinel/toolboxes/sentinel-1)__
   - __[ISCE](https://github.com/isce-framework)__ is a general-purpose processing software for InSAR developed by folks at JPL and elsewhere, written in Python
   - __[GMTSAR](https://topex.ucsd.edu/gmtsar/)__ is a general InSAR processing tool developed by folks at Scripps Institute of Oceanography, writting in the GMT language
   - __[STAMPS](https://github.com/dbekaert/StaMPS)__ for advanced time-series analysis
   - Various tools for atmospheric corrections: __[TRAIN](https://github.com/dbekaert/TRAIN)__, __[GACOS]()__, __[RAiDER](https://github.com/dbekaert/RAiDER)__ (which I have worked on), and __[PyAPS](http://earthdef.caltech.edu/projects/pyaps/wiki/Main)__.
   - __[MintPy](https://github.com/insarlab/MintPy)__ for InSAR time-series analysis (__[Mintpy tutorial)](https://github.com/insarlab/MintPy-tutorial)__
   - __[PyRate](https://github.com/GeoscienceAustralia/PyRate)__ is a Python tool for InSAR time-series analysis
   - __[GIAnT](http://earthdef.caltech.edu/projects/giant/wiki)__ is yet another InSAR time-series package written in Python
   
__[UNAVCO](https://www.unavco.org/)__ has a number of __[educational resources](https://www.unavco.org/education/education.html)__ available for learning to use the general-purpose InSAR processors, including a __[short courses](https://www.unavco.org/education/professional-development/short-courses/short-courses.html)__, one of which will be __[taught this year](https://www.unavco.org/education/professional-development/short-courses/2020/insar-theory-processing/insar-theory-processing.html)__, and __[past courses](https://www.unavco.org/education/professional-development/short-courses/course-materials/course-materials.html)__ as well. 

<img src="https://raw.githubusercontent.com/jlmaurer/GE6146/master/notebooks/images/UNAVCO_homescreen.png" alt="UNAVCO home page" width="700">

For this lab we will use a __[shared Google Drive folder](https://drive.google.com/drive/folders/1hm3FD0vQXbdvaR2wMXg3GC9AYgYTYBj6?usp=sharing)__ so that you do not need to separately download data. I have already download the data required for this lab to the data/ folder inside the linked shared folder, so you can directly access that. Python programs required for this lab are also in the shared folder so that they are accessible to us while we go through the lab. 

**Each of you have a read/write/edit Google folder inside the shared folder, where you can save intermediate products or results, or even save a copy of this notebook. If you need to save or write data during this lab, use this folder!**

In [2]:
from google.colab import drive
drive.mount('/gdrive')

ModuleNotFoundError: No module named 'google'

In [8]:
# Change to the ARIAtools directory on the shared Google Drive
%cd /gdrive/My\ Drive/GE6371/

In [ ]:
# Import some libraries to be used later
import os
import numpy as np
from osgeo import gdal, ogr

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, FormatStrFormatter, StrMethodFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

!pip install ARIAtools/ARIA-tools

## 1. InSAR processing with ARIA products
**Authors**: Brett A. Buzzanga, David Bekaert, Simran Sangha - Jet Propulsion Laboratory, Jeremy Maurer - Missouri S&T

In this notebook we will learn how to use the ARIA family of command line tools to access Sentinel 1 ARIA Geocoded UNWrapped interferogram (**GUNW**) products.  A detailed overview of the ARIA GUNW product with respect to processing, formatting, sampling, and data layers can be found on the __[ARIA website](https://aria.jpl.nasa.gov/node/97)__. This notebook has been modified from the __[ARIA-tool-docs](https://github.com/aria-tools/ARIA-tools-docs)__ repository, which holds the original notebooks. The ARIA-tools code can be found at the __[Github repository](https://github.com/aria-tools/ARIA-tools)__. 

### Downloading GUNW products using ariaDownload.py
The **`ariaDownload.py`** program wraps around the NASA's ASF DAAC API and __[Bulk Download Service](https://bulk-download.asf.alaska.edu/help)__. The ASF Bulk Download Service handles most of the heavy lifting of the data-download and will conveniently skip previously downloaded files, and re-download partially downloaded files.  
In this notebook, we will demonstrate **`ariaDownload.py`** functionality along track 4, which intersects the U.S. East Coast in southeastern Virginia.

<div class="alert alert-warning">
<b>Potential download failure:</b>
GUNW products are hosted at the NASA ASF DAAC. Downloading them requires a NASA Earthdata URS user login and requires users to add “ARIA Product Search” to their URS approved applications
</div>

<div class="alert alert-danger">
<b>Download</b>:     
    
- The ***jupyter notebook* does not allow for interactive entering of your user-name and password, use the *jupyter terminal* instead** with the same command for interactive use.
</div>

Running **`ariaDownload.py`** with no options, or with **`-h`**, will show the parameters options as well as some examples. At minimum, users need to specify a spatial constraint either as a track number or bounding box (can be a shapefile).

Let us explore what some of the other options are

In [12]:
# Display the help message for ariaDownload.py
!ariaDownload.py -h

usage: ariaDownload.py [-h] [-o OUTPUT] [-t TRACK] [-b BBOX] [-w WD]
                       [-s START] [-e END] [-l DAYSLT] [-m DAYSGT] [-i IFG]
                       [-d FLIGHTDIR] [-v]

Command line interface to download GUNW products from the ASF DAAC. GUNW products are hosted at the NASA ASF DAAC.

optional arguments:
  -h, --help            show this help message and exit
  -o OUTPUT, --output OUTPUT
                        Output type, default is "Download". "Download",
                        "Count", "Url" and "Kmz" are currently supported. Use
                        "Url" for ingestion to aria*.py
  -t TRACK, --track TRACK
                        track to download; single number (including leading
                        zeros) or comma separated
  -b BBOX, --bbox BBOX  Lat/Lon Bounding SNWE, or GDAL-readable file
                        containing POLYGON geometry.
  -w WD, --workdir WD   Specify directory to deposit all outputs. Default is
                        "products

**Count the number of products**  
To get a count of the number of products, without downloading data, provide the **`--output`** option with the **`count`** argument. To get information on the exact product filenames also include the verbose option **`-v`**.

In [ ]:
# Check the number of acquisitions for track 4
!ariaDownload.py --track 4 --output count

**Generate list of virtual products from ASF S3 bucket (BETA)**  
To generate a textfile containing a list of product URLs from the ASF S3 bucket, without downloading data, provide the **`--output`** option with the **`url`** argument. To get information on the exact product filenames also include the verbose option **`-v`**. Extracting layers virtually by leveraging this list of URLs is currently only supported by systems with the following packages: Linux kernel >=4.3 and libnetcdf >=4.5 

In [ ]:
# Write a file to your writeable folder in Google drive using the --output url
!ariaDownload.py --track 4 --output url -w <your_writeable_folder>

**Generate KMZ of the products**  
To generate a Google Earth kmz, without downloading data, provide the **`--output`** option with the **`kmz`** argument. In case you also want to get information on the exact product filenames you can enbale the verbose option by adding **`-v`**. The result is shown in **Fig. 1**.

In [ ]:
# make sure to specify the correct folder to write to using the -w option
!ariaDownload.py --track 4 --output kmz -w <your_writeable_folder>

**Download the data**  
To download the products, simply omit the **`--output`** option or specify it with the **`download`** argument. Examples for downloading all data on track 4 include:

`ariaDownload.py -t 004`

`ariaDownload.py -t 004 -o download`


By default, the products will be downloaded into the *./products* folder. You can specify a different location using the **`-w`** option, such as:

`ariaDownload.py -t 004 -w /insarHome`

<div class="alert alert-warning">
<b>Potential download failure:</b>
GUNW products are hosted at the NASA ASF DAAC. Downloading them requires a NASA Earthdata URS user login and requires users to add “ARIA Product Search” to their URS approved applications
</div>

<div class="alert alert-danger">
<b>Download</b>: 
- You will not need to actually download any data for this lab; I have already downloaded the data and made it available in the data/ folder in the shared Google drive. However, you will need to download data to your own location if you use InSAR for your final project. 
</div>

**Spatial-temporal subsetting**  
The **`ariaDownload.py`** program has a number of options availble for subsetting the search of products in the spatial and temporal domain, including **bounding box**, **start/end date**, **aquisition geometry**, **temporal baseline**, or **interferogram** options. Below, using the count option for brevity, we demonstrate each of them.

**Specify a bounding box**  
Bounding box can either be specified as **SNWE** coordinates (a string with quotation) or by providing a **shapefile** (GeoJSON or ESRI shapefiles) to the **`--bbox`** option. 

In [ ]:
# An example for track 4 over eastern US
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count

**Subsetting by start/end date**  
Subsetting in time can be done by specifying a range within which products need to fall. This is controlled using the **`--start YYYYMMDD` ** and **`--end YYYYMMDD`** options. By default the complete observational record is considered.

In [ ]:
!ariaDownload.py --track 4 --bbox "36.75 37.225 -76.655 -75.928" -o count -s 20190101  --end 20190401 --verbose

**Specifying by a specific interferogram** 
To find a specific interferogram combination one can use the **`--ifg YYYYMMDD_YYYYMMDD`** option.

In [ ]:
!ariaDownload.py -b "36.75 37.225 -76.655 -75.928"  -o count --ifg "20161018_20160702" -v 

### An example: Kilauea volcano, Hawaii
In this lab we will look at some data covering the Big Island of Hawaii, during early 2018 when the Kilauea volcano was erupting. (You can read more about the eruption __[here](https://www.usgs.gov/news/k-lauea-volcano-erupts)__. We will use ARIA data from the time period during the eruption to analyze deformation during the event. 

In [9]:
# Check how many acquisitions are available from January to July 2018 over Hawaii, when Kilauea volcano was erupting on the Big Island
!ariaDownload.py --track 124 -b '18.8 20.3 -156.2 -154.6' -d ascending -s 20180101 -e 20180701  --output count

python: can't open file 'ARIA-tools/tools/bin/ariaDownload.py': [Errno 2] No such file or directory


In [ ]:
# I've already downloaded the data to the Google Drive, look at it by running this cell
!ls data/

## 2. Plotting the data with ariaPlot.py
The **ariaPlot.py** program allows for easy generation of qualitative and spatiotemporal coverage plots of products over a user-defined area of interest. Running **ariaPlot.py** with the **-h** option, will show the parameters options. 

Let us explore these options:

In [1]:
# ariaPlot.py help menu
!ariaPlot.py -h

python: can't open file 'ARIA-tools/tools/bin/ariaPlot.py': [Errno 2] No such file or directory


**Specifying which products to use**  
At minimum, users need to specify the GUNW files they want to extract information from. This is controlled using the **`-f`** option. Multiple products can be specified by providing them as a comma separated string (e.g., **`-f`**` 'products/S1-GUNW-D-R-042-tops-20150605_20150512-140722-39616N_37642N-PP-e396-v2_0_0.nc,products/S1-GUNW-D-R-042-tops-20150629_20150512-140723-39615N_37641N-PP-0e95-v2_0_0.nc'`), or using a wildcard (e.g., **`-f`**` 'products/S1*.nc'`).

**Cropping and spatial subsetting (-b and -croptounion options)**  
The **`ariaPlot.py`** program will automatically handle cropping and stitching of GUNW products when needed. By default, the program will crop all interferograms to bounds determined by the common intersection (of all interferograms) and the user-defined bounding box option. All layers are cropped and/or stitched using GDAL (see the methods section for details on the implemented approach for each layer). Below we discuss which options are available for specifying an area-of-interest.

GUNW products are grouped in clusters that belong to the same interferometric pair. By default, the spatial **intersection** of the interferometic pairs is used to define the region of interest. This can be overriden to be the union of all interferograms (regardless of alignment) by passing the **`--croptounion`** argument. A schematic example is shown in **Fig 1** for both scenario's.

<div class="alert alert-warning">
<b>Warning:</b> Users in general should avoid mixing products of adjacent satellite tracks (i.e., products made on the same contiguous pass are ok). Note that along the equator, the track number (ascending data on the ascending note) gets incremented while the data itself is still continuous.
</div>

<img src="https://raw.githubusercontent.com/jlmaurer/GE6146/master/notebooks/images/spatial_config_new.png" alt="spatial_configuration" width="700">
<blockquote> <center><b> Fig. 1 </b> Schematic of the spatiotemporal configuration for three interferograms. The left panel shows the intersection of the interferograms, which is the default behavior. The right panel shows the union of the interferograms, achieved by passing <b><code>--croptounion</code></b> . The blue dashed line demonstrates the behavior if the user had specified a bounding box (<b><code>-b</code></b>). Note that interferograms which do not cover the bounding box completely with the <b><code>--croptounion</code></b> or the <b><code>-b</code></b> option will be patched with no-data values.</center></blockquote>

Users can analyze the interferogram quality over their study area using the interferogram baseline and average coherence information. **`ariaPlot.py`** allows users to visualize this information directly from the GUNW products. Three main options are available that can assist users for a qualitative assessment. The **`--plotbperpcoh`** option generates a perpendicular baseline plot over time, color-coded with average coherence. An alternative representation is provided by the **`--plotcoh`** option, which shows just the average interferogram coherence over time. Lastly, the **`--makeavgoh`** option can be leveraged for making a 2D product of average coherence in time. 

### Generating plots in ARIA-tools
The following commented-out command would generate the plots described in the last section. I have already generated the plots and you can see them in the /plots folder. Open the plots and look at them on your local computer. 

In [ ]:
# Create a "figues" directory inside your writeable directory and replace <your_user_name> 
# with your writeable directory name, for example: jlmd9g/figures. 
#!ariaPlot.py -f "data/*.nc"   --plotcoh --plotbperpcoh --makeavgoh -w <your_user_name>/figures>

In [10]:
# Look at the files in your figures directory. You can open these on directly from your Google Drive 
# (not in this notebook)
!ls plots

ls: figures: No such file or directory


In [ ]:
# You can use this function as is (do not need to modify it)
def plot_layer(path_layer, lay_type=None, cmap=None, **kwargs):
    """ 
        path_layers is a string to the GDAL compatible dataset to be plotted
    """
    
    if not lay_type: 
        lay_type = os.path.dirname(path_layer)
    title = [os.path.basename(lay_type)]
    
    ## get the lon lat bounds
    ds       = gdal.Open(path_layer, gdal.GA_ReadOnly)
    trans    = ds.GetGeoTransform()
    extent   = [trans[0], trans[0] + ds.RasterXSize * trans[1], trans[3] + ds.RasterYSize*trans[5], trans[3]]
    
    ## loading the data
    n_bands  = ds.RasterCount
    lst_arrs = []
    
    for band in range(n_bands):
        raster = ds.GetRasterBand(band+1)
        arr    = raster.ReadAsArray()
        try:
            NoData = raster.GetNoDataValue()
            arr = np.ma.masked_where((arr>1e20) |(arr==NoData),arr )
        except:
            print('Could not find a no-data value...')
            arr = np.ma.masked_where(arr>1e20,arr)
        
        lst_arrs.append(arr)

    ds = None
    if n_bands < 4:
        nrows = 1; ncols = n_bands
    else:
        raise Exception('Number of bands currently unsupported')
        
    
    ## initializing a figure
    fig, axes = plt.subplots(figsize=(12,9), ncols=ncols, nrows=nrows, sharex='col', sharey='row')
    axes = axes if isinstance(axes, np.ndarray) else np.array(axes)
    axe  = axes.ravel() 
    cmap = plt.cm.Greys_r
    cmap.set_under('black')
    
    ## definging the plotting options for differnt layer types
    # Amplitude:
    if lay_type.endswith('amplitude'): 
        # will fix the maximum amplitude bound
        vmin=None
        vmax = 2000 
    # Coherence:
    elif lay_type.endswith('coherence'): 
        # has fixed range between 0-1
        vmin=0
        vmax = 1
    # Incidence angle:
    elif lay_type.endswith('incidenceAngle'): 
        vmin=None
        vmax=None
    # water
    elif lay_type.startswith('water'):
        # no bounds needed will be a 0/1 mask
        vmin=None
        vmax=None
    # deformation or unwrapped phase
    elif lay_type.startswith('defo'): 
        # let the data drive the bounds
        vmin=None
        vmax=None
        # change colormap to a warm type
        cmap=plt.cm.coolwarm
    elif lay_type.startswith('terr') or lay_type.startswith('topo'): 
        # let the data drive the bounds
        vmin=None
        vmax=None
        # change colormap to a warm type
        cmap=plt.cm.terrain
    elif lay_type == 'ENU':
        vmin=None
        vmax=None
        title = ['East', 'North', 'Up']
        fig.subplots_adjust(wspace=0.5)

    else:
        # let the data drive the bounds
        vmin=None
        vmax=None
        # change colormap to a warm type
        cmap=plt.cm.coolwarm
        
    # plotting the data    
    for i, ax in enumerate(axe):
        im   = ax.imshow(lst_arrs[i], cmap=cmap, vmin=vmin, vmax=vmax, extent=extent)
        divider = make_axes_locatable(ax)
        cax     = divider.append_axes('right', size='5%', pad=0.25)
        if lay_type == 'ENU':
            fig.colorbar(im, cax=cax, format=FuncFormatter(lambda x, y: '{:.3f}'.format(x)))
        else:
            fig.colorbar(im, cax=cax)

        ax.set_title(title[i], fontsize=15)
        ax.grid(False)

    axe[0].set_ylabel('latitude', labelpad=15, fontsize=15)
    axe[int(np.floor(n_bands/2))].set_xlabel('longitude', labelpad=15, fontsize=15)


## 3. Preparing ARIA standard GUNW products layers for time-series analysis using ariaTSsetup.py

**Author**: Simran Sangha, David Bekaert - Jet Propulsion Laboratory

This notebook provides an overview of the functionality included in the **ariaTSsetup.py** program. Specifically, we give examples on how to extract data and meta-data layers from ARIA Geocoded UNWrapped interferogram (GUNW) products over a user defined area of interest and prepare the data into a stack for time-series ingestion.

In this notebook, we will demonstrate how to:
- Extract data layers (unwrapped phase, coherence) and imaging geometry layers (azimuth angle, incidence angle, look angle) necessary for building time-series
- Prepare the extracted data into a stack for time-series ingestion

    
<div class="alert alert-warning">
Both the initial setup (<b>Prep A</b> section) and download of the data (<b>Prep B</b> section) should be run at the start of the notebook. The overview sections do not need to be run in order. In the application section the ariaTSsetup commandline call at the top must be run first, but the rest of the section does not need to be run in order.
</div>

<div class="alert alert-danger">
<b>Potential Errors:</b> 
    
- GDAL uses "HDF5" driver instead of "netCDF/Network Common Data Format" on GUNW products. Verify GDAL version >= 3.
- ARIA-tools needs to be installed to run this notebook
</div>


<div class="alert alert-info">
    <b>Terminology:</b>
    
- *Acquisition*: An image acquired by the satellite for a given date and time.
- *Interferogram*: An unwrapped image containing the surface displacement accumulated between two acquisitions.
- *Frame*: Outline of a product ground footprint.
- *Along-track*: The direction along satellite flight path. 
    </div>
    

In [ ]:
# look at the help menu
!ariaTSsetup.py -h

The following command would be used to generate the products that will go into the time-series analysis. I've already processed the data and made it available in my folder, /jlmd9g. 

In [ ]:
# Prepare the interferograms for time-series analysis
# Make sure to specify your username directory!
# !ariaTSsetup.py -f 'data/*.nc' -b '18.8 20.3 -156.2 -154.6' -d Download --mask Download -w <your_user_name>

In [ ]:
!ls jlmd9g

In [ ]:
# There is a water mask generated that will be used to ignore pixels covering the ocean
plot_layer('jlmd9g/mask/watermask.msk',lay_type='water')

In [ ]:
# Look at the downloaded DEM
plot_layer('jlmd9g/DEM/SRTM_3arcsec.dem',lay_type='topo')

In [ ]:
# Look at the coherence of one of the acquisitions
plot_layer('jlmd9g/coherence/20180114_20180108',lay_type='coherence')

## 4. InSAR time-series analysis
For looking at earthquakes and other large deformation, often one or two interferograms will be sufficient to see the deformation. For smaller or long-term processes, InSAR time-series must be used. This introduces some more complexity into the processing because of the need to co-register interferograms and account for various noise sources. In the next example, you will use a simple InSAR time-series generator to look at deformation during the 2018 Kilauea volcano eruption on the Big Island of Hawaii. 

The 2018 Kilauea eruption resulted in uplift at the volcano because of extrusion of lava and injection of magma into the shallow subsurface, but the summit of the mountain deflated because magma moved from under the summit to the flank. You can read more about it __[here](https://www.usgs.gov/news/k-lauea-volcano-erupts)__. 

You will use the functions in "makeTS.py" to create a velocity map using the ARIA interferograms that I've downloaded. I've converted them to an aligned stack in the directory jlmd9g/interferogram_stack 

In [ ]:
!ls jlmd9g/unwrappedPhase

In [ ]:
# makeTS is a module containing several functions, including dt2fracYear. We'll import them all!
!cp jlmd9g/makeTS.py .
from makeTS import *  # NOTE: generally this syntax is not a good idea, but for this small package it's ok
ifgList = glob.glob('jlmd9g/unwrappedPhase/*.vrt')

In [ ]:
# print the list of interferograms
# Your code here

In [ ]:
# First we will get the list of dates
datePairs, dates = getDates(ifgList)

In [ ]:
# Look at the date pairs and dates
# what is the difference? Which one relates to interferograms and which relates to time-series? 
# Your answers here

In [ ]:
# Next, use the dt2fracYear function to convert python datetimes to fractional dates
fracDates = np.array([dt2fracYear(d) for d in dates])

## Interferograms to time-series
To estimate the displacement at each date, we use a G-matrix. This means that we will solve a linear system: 
**I = Gd**, where **I** is the stack of interferograms (actually, there is one stack for each pixel in each interferogram), **G** is called the "G-matrix" (or model matrix), and **d** is the displacement between each acquisition time and the one before. 

The columns of the **G**-matrix correspond to each acqiusition date, and the rows correspond to each interferogram. The elements of this matrix are simple: each interferogram gets a row, with a "-1" at the location of the second acquisition date, and a "+1" at the first acquisition date. Something like this:  
row 1: [0 0 0 0 -1 1 0 0]  
row 2: [0 0 0 0 0 0 -1 1]  
row 3: [-1 0 1 0 0 0 0 0]  
row 4: [-1 0 0 1 0 0 0 0]  
would represent a potential G-matrix. In this case, we have fewer interferograms then dates, but in most cases there will be more. We have to have, at a minimum, one interferogram using every acquisition date. 

For those of you familiar with linear algebra and inverse theory, this G-matrix is inverted to solve for the displacements at each time period. Since we have more interferograms than acquisition times, this is an _over-determined_ system, and we will solve for the best-fitting time-series that matches all of the interferograms. We could then go back to the interfergrams by differencing the displacements between the two corresponding acquisition times. 

In [ ]:
# G itself is not related to the data; it only depends on the model (in this case, which dates are linked by which interferograms)
G = makeG(dates, datePairs)

In [ ]:
# get the details about the size of the interferogram raster images: size, projection, and so forth
xSize, ySize, dType, geoProj, trans, noDataVal, Nbands = readRaster(ifgList[0])

In [ ]:
# read all the interferograms into memory
data = getData(ifgList,1)
print(data.shape) # this should be a big 3-d matrix with each interferogram

## Reference region
**InSAR is a relative measurement.** This is because the measurements are modulo $2\pi$, so we never know what the absolute displacements are without comparing to some other measurement, such as GPS. In order to create a time-series, we have to relate all the interferograms, so we have to create a "reference point" (or reference region) where all the interferograms are identical. The only way to do this is to subtract out the value of the reference point from each unwrapped interferogram **before** estimating the time-series. This is what "dereference" does in the code below. Passing "None" to refCenter will use the center of the image, but you can adjust this to fit your preferences if needed. The refCenter pixel will have zero displacement in the time-series, so you want to pick an area that is not deforming. 

In [ ]:
data = dereference(data, taxis=0,refCenter=None,refSize=10)

With the interferograms referenced to a common region, and the G-matrix for relating dates and interferograms, we can now solve for displacements through time!

In [ ]:
# Go from interferograms to time-series
tsArray = makeTS(G, data, fracDates)

### Note: 
The native InSAR displacements are in radians. To convert to meters, we need to multiply the displacements by $\frac{\lambda}{-4\pi}$, where $\lambda$ is the radar wavelength (=0.056 meters, or 5.6 cm, for Sentinel 1). This assumes "date_2 - date_1". When we do "date_1 - date_2", then we don't use the negative sign: $\frac{\lambda}{4\pi}$. ARIA products use the "date_2 - date_1" convention. 

In [ ]:
# The data is still in phase (radians) so we'll convert to meters
tsArray = convertRad2meters(tsArray, lam=0.056) # wavelength lam is 5.6 cm for Sentinel-1

The plot below will compare a pixel located on the volcano to one on the summit of the mountain caldera. During the eruption, the volcano inflated and erupted while the summit actually deflated due to magma moving out the side. 

In [ ]:
# We plot single pixels through time
plt.plot(fracDates, tsArray[:,1100,1300])  # this is a pixel on the Kilauea volcano
plt.plot(fracDates, tsArray[:,700,1000])  # this is a pixel on the summit of Moana Loa
plt.xlabel('Year')
plt.ylabel('Displacement (m)')
plt.show()

## Velocity estimation
We can use the time-series to estimate various parameters of the event, such as the total or average rate of displacement. These can then be used to infer properties of the eruption, such as the volume of erupted fluid. We will estimate the mean line-of-sight velocity below. **Remember** that the line-of-sight for a satellite is mainly sensitive to vertical motion, so most of the displacements are vertical.

### Note:
InSAR displacements change sign depending on the order you create the interferograms. Each interferogram has two images that are differenced, so you can do "date_1 - date_2" _or_ "date_2 - date_1". 

In [ ]:
# Compute the mean velocity by fitting a line
vel = findMeanVel(tsArray, fracDates, 0)

In [ ]:
# Now we can plot the mean velocity throughout the period
cax = plt.imshow(vel, vmax=None,vmin=None) # vmax and vmin adjusts the minimum and maximum color shown
plt.colorbar()
plt.show()

In [ ]:
# save the data to an HDF5 file for use later
filename='ts.h5' # create your own filename with the path to your writeable folder, for example, 'jlmd9g/ts.h5'
writeTS2HDF5(tsArray, fracDates,vel,filename='ts.h5')

When turning in your notebook, make sure to use "plt.show()" in each cell with a plot so that it will show for me when I open your turned-in version. 